<a href="https://colab.research.google.com/github/venezianof/booksum/blob/main/colabs/AutoTrain_ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title 🤗 AutoTrain
#@markdown In order to use this colab
#@markdown - Enter your [Hugging Face Write Token](https://huggingface.co/settings/tokens)
#@markdown - Enter your [ngrok auth token](https://dashboard.ngrok.com/get-started/your-authtoken)
huggingface_token = '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 ' # @param {type:"string"}
ngrok_token = "2u9v0BA9aZJFyqeRmLGPjyyVOBa_PFyoMKxBsUrmB8quxAq2" # @param {type:"string"}

#@markdown
#@markdown - Attach appropriate accelerator `Runtime > Change runtime type > Hardware accelerator`
#@markdown - click `Runtime > Run all`
#@markdown - Follow the link to access the UI
#@markdown - Training happens inside this Google Colab
#@markdown - report issues / feature requests [here](https://github.com/huggingface/autotrain-advanced/issues)

import os
os.environ["HF_TOKEN"] = str(huggingface_token)
os.environ["NGROK_AUTH_TOKEN"] = str(ngrok_token)
os.environ["AUTOTRAIN_LOCAL"] = "1"

!pip install -U autotrain-advanced > install_logs.txt 2>&1
!autotrain app --share

INFO     | 2025-04-02 20:17:56 | autotrain.cli.run_app:run:132 - AutoTrain Public URL: NgrokTunnel: "https://5032-34-125-26-254.ngrok-free.app" -> "http://localhost:7860"
INFO     | 2025-04-02 20:17:56 | autotrain.cli.run_app:run:133 - Please wait for the app to load...
INFO     | 2025-04-02 20:18:02 | autotrain.app.ui_routes:<module>:31 - Starting AutoTrain...
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_http.py", line 406, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/models?filter=transformers&filter=fill-mask&sort=downloads&direction=-1&limit=100

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local

In [2]:
!pip install smolagents

  Using cached huggingface_hub-0.30.1-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.30.1-py3-none-any.whl (481 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autotrain-advanced 0.8.36 requires huggingface-hub==0.27.0, but you have huggingface-hub 0.30.1 which is incompatible.


In [3]:
from smolagents import DuckDuckGoSearchTool

In [4]:
search_tool = DuckDuckGoSearchTool()

In [5]:
results = search_tool("Who's the current President of France?")
   print(results)

IndentationError: unexpected indent (<ipython-input-5-0bd2e379901c>, line 2)

In [ ]:
!pip install llama-index huggingface_hub

In [ ]:
from llama_index.core.agent.workflow import AgentWorkflow
   from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

   # Initialize the Hugging Face model
   llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")

   # Define your tools (search_tool, weather_info_tool, hub_stats_tool)
   # ... (Refer to previous responses for definitions) ...

In [ ]:
!pip install langgraph langchain-huggingface

In [ ]:
from typing import TypedDict, Annotated
   from langgraph.graph.message import add_messages
   from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
   from langgraph.prebuilt import ToolNode
   from langgraph.graph import START, StateGraph
   from langgraph.prebuilt import tools_condition
   from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

   # Generate the chat interface, including the tools
   # Replace 'YOUR_HUGGINGFACEHUB_API_TOKEN' with your actual token
   HUGGINGFACEHUB_API_TOKEN = 'YOUR_HUGGINGFACEHUB_API_TOKEN'
   llm = HuggingFaceEndpoint(
       repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
       huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
   )

   chat = ChatHuggingFace(llm=llm, verbose=True)
   tools = [search_tool, weather_info_tool, hub_stats_tool]  # Define your tools as before
   chat_with_tools = chat.bind_tools(tools)

In [ ]:
# Generate the AgentState and Agent graph
   class AgentState(TypedDict):
       messages: Annotated[list[AnyMessage], add_messages]

   def assistant(state: AgentState):
       return {
           "messages": [chat_with_tools.invoke(state["messages"])],
       }

   ## The graph
   builder = StateGraph(AgentState)

   # Define nodes: these do the work
   builder.add_node("assistant", assistant)
   builder.add_node("tools", ToolNode(tools))

   # Define edges: these determine how the control flow moves
   builder.add_edge(START, "assistant")
   builder.add_conditional_edges(
       "assistant",
       # If the latest message requires a tool, route to tools
       # Otherwise, provide a direct response
       tools_condition,
   )
   builder.add_edge("tools", "assistant")
   alfred = builder.compile()

In [ ]:
messages = [HumanMessage(content="Who is Facebook and what's their most popular model?")]
   response = alfred.invoke({"messages": messages})

   print("🎩 Alfred's Response:")
   print(response['messages'][-1].content)

In [ ]:
# DuckDuckGoSearchTool
   from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
   from llama_index.core.tools import FunctionTool

   tool_spec = DuckDuckGoSearchToolSpec()
   search_tool = FunctionTool.from_defaults(tool_spec.duckduckgo_full_search)

   # WeatherInfoTool
   import random
   from llama_index.core.tools import FunctionTool

   def get_weather_info(location: str) -> str:
       """Fetches dummy weather information for a given location."""
       weather_conditions = [
           {"condition": "Rainy", "temp_c": 15},
           {"condition": "Clear", "temp_c": 25},
           {"condition": "Windy", "temp_c": 20}
       ]
       data = random.choice(weather_conditions)


In [ ]:
# Create Alfred with all the tools
   alfred = AgentWorkflow.from_tools_or_functions(
       [search_tool, weather_info_tool, hub_stats_tool],
       llm=llm
   )

In [ ]:
# Example query Alfred might receive during the gala
   response = await alfred.arun("What is Facebook and what's their most popular model?")  # Use arun for async

   print("🎩 Alfred's Response:")
   print(response)

In [ ]:
!pip install smolagents huggingface_hub

In [ ]:
from smolagents import CodeAgent, HfApiModel, Tool
   from huggingface_hub import list_models
   import random

   # Initialize the Hugging Face model
   model = HfApiModel()

   # Define and initialize your tools (search_tool, weather_info_tool, hub_stats_tool)
   # You'll need to define and initialize these tools as you did in the previous examples.
   # ... (Code for the tools as provided in earlier responses) ...

In [ ]:
# DuckDuckGoSearchTool
from smolagents import DuckDuckGoSearchTool
search_tool = DuckDuckGoSearchTool()

# WeatherInfoTool
class WeatherInfoTool(Tool):
    name = "weather_info"
    description = "Fetches dummy weather information for a given location."
    inputs = {
        "location": {
            "type": "string",
            "description": "The location to get weather information for."
        }
    }
    output_type = "string"

    def forward(self, location: str):
        # Dummy weather data
        weather_conditions = [
            {"condition": "Rainy", "temp_c": 15},
            {"condition": "Clear", "temp_c": 25},
            {"condition": "Windy", "temp_c": 20}
        ]
        # Randomly select a weather condition
        data = random.choice(weather_conditions)
        return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"
weather_info_tool = WeatherInfoTool()

# HubStatsTool
class HubStatsTool(Tool):
    name = "hub_stats"
    description = "

In [ ]:
# Create Alfred with all the tools
   alfred = CodeAgent(
       tools=[search_tool, weather_info_tool, hub_stats_tool],
       model=model
   )

In [ ]:
# Example query Alfred might receive during the gala
   response = alfred.run("What is Facebook and what's their most popular model?")

   print("🎩 Alfred's Response:")
   print(response)

In [ ]:
!pip install smolagents huggingface_hub

In [ ]:
!pip install llama-index huggingface_hub

In [ ]:
import random
   from llama_index.core.tools import FunctionTool
   from huggingface_hub import list_models

   def get_hub_stats(author: str) -> str:
       """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
       try:
           # List models from the specified author, sorted by downloads
           models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

           if models:
               model = models[0]
               return f"The most downloaded model by {author} is {model.id} with {model.downloads

In [ ]:
# Initialize the tool
   hub_stats_tool = FunctionTool.from_defaults(get_hub_stats)

In [ ]:
# Example usage
   print(hub_stats_tool.invoke("facebook"))  # Example: Get the most downloaded model by Facebook

In [ ]:
!pip install langchain huggingface_hub

In [ ]:
from langchain.tools import Tool
from huggingface_hub import list_models

def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
    try:
        # List models from the specified author, sorted by downloads
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models:
            model = models[0]
            return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"

In [ ]:
# Initialize the tool
hub_stats_tool = Tool(
    name="get_hub_stats",
    func=get_hub_stats,
    description="Fetches the most downloaded model from a specific author on the Hugging Face Hub."
)

In [ ]:
# Example usage
print(hub_stats_tool.run("facebook")) # Example: Get the most downloaded model by Facebook

In [ ]:
!pip install langchain_community

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

In [ ]:
search_tool = DuckDuckGoSearchRun()

In [ ]:
results = search_tool.invoke("Who's the current President of France?")
   print(results)

In [ ]:
!pip install smolagents

In [ ]:
from smolagents import Tool
import random

class WeatherInfoTool(Tool):
    name = "weather_info"
    description = "Fetches dummy weather information for a given location."
    inputs = {
        "location": {
            "type": "string",
            "description": "The location to get weather information for."
        }
    }
    output_type = "string"

    def forward(self, location: str):
        # Dummy weather data
        weather_conditions = [
            {"condition": "Rainy", "temp_c": 15},
            {"condition": "Clear", "temp_c": 25},
            {"condition": "Windy",

In [ ]:
weather_info_tool = WeatherInfoTool()

In [ ]:
result = weather_info_tool.forward("Paris")
    print(result)

In [ ]:
!pip install smolagents

In [ ]:
from smolagents import Tool
import random

class WeatherInfoTool(Tool):
    name = "weather_info"
    description = "Fetches dummy weather information for a given location."
    inputs = {
        "location": {
            "type": "string",
            "description": "The location to get weather information for."
        }
    }
    output_type = "string"

    def forward(self, location: str):
        # Dummy weather data
        weather_conditions = [
            {"condition": "Rainy", "temp_c": 15},
            {"condition": "Clear", "temp_c": 25},
            {"condition": "Windy", "temp_c": 20}
        ]
        # Randomly select a weather condition
        data = random.choice(weather_conditions)
        return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

In [ ]:
weather_info_tool = WeatherInfoTool()

In [ ]:
result = weather_info_tool.forward("Paris")
    print(result)

In [ ]:
import random
from llama_index.core.tools import FunctionTool

def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""
    # Dummy weather data
    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Clear", "temp_c": 25},
        {"condition": "Windy", "temp_c": 20}
    ]
    # Randomly select a weather condition
    data = random.choice(weather_conditions)
    return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

In [ ]:
weather_info_tool = FunctionTool.from_defaults(get_weather_info)

In [ ]:
# Make a prediction using the tool
weather_info_tool.invoke("Paris")

In [ ]:
!pip install langchain

In [ ]:
import random
   from langchain.tools import Tool

   def get_weather_info(location: str) -> str:
       """Fetches dummy weather information for a given location."""
       # Dummy weather data
       weather_conditions = [
           {"condition": "Rainy", "temp_c": 15},
           {"condition": "Clear", "temp_c": 25},
           {"condition": "Windy", "temp_c": 20}
       ]
       # Randomly select a weather condition
       data = random.choice(weather_conditions)
       return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C

In [ ]:
# Initialize the tool
   weather_info_tool = Tool(
       name="get_weather_info",
       func=get_weather_info,
       description="Fetches dummy weather information for a given location."
   )

In [ ]:
result = weather_info_tool.run("Paris")
    print(result)

In [ ]:
import random
from llama_index.core.tools import FunctionTool

def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""
    # Dummy weather data
    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Clear", "temp_c": 25},
        {"condition": "Windy", "temp_c": 20}
    ]
    # Randomly select a weather condition
    data = random.choice(weather_conditions)
    return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

In [ ]:
weather_info_tool = FunctionTool.from_defaults(get_weather_info)

In [ ]:
# Make a prediction using the tool
weather_info_tool.invoke("Paris")

In [ ]:
!pip install langchain_community

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

In [ ]:
search_tool = DuckDuckGoSearchRun()

In [ ]:
results = search_tool.invoke("Who's the current President of France?")
   print(results)

In [ ]:
!pip install llama-index

In [ ]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
   from llama_index.core.tools import FunctionTool

In [ ]:
tool_spec = DuckDuckGoSearchToolSpec()
   search_tool = FunctionTool.from_defaults(tool_spec.duckduckgo_full_search)

In [ ]:
response = search_tool("Who's the current President of France?")
   print(response.raw_output[-1]['body'])  # Access the body of the last result